In [1]:
%cd ..
%load_ext nb_black

/home/hardianlawi/attention-lstm-sentiment-analysis


<IPython.core.display.Javascript object>

In [2]:
from src.preprocess import Preprocessor
from src.data import get_data, ID2WORD, OOV_ID, START_ID, WORD2ID


<IPython.core.display.Javascript object>

In [3]:
vocabulary_size = 5000
embedding_size = 32
max_words = 500

(X_train, y_train), (X_test, y_test) = get_data(vocabulary_size)

print("Maximum review length: {}".format(len(max((X_train + X_test), key=len))))
print("Minimum review length: {}".format(len(min((X_train + X_test), key=len))))


Maximum review length: 2494
Minimum review length: 7


<IPython.core.display.Javascript object>

In [4]:
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras import Model


def _generate_elmo_embeddings():
    temp = X_train + X_test
    temp = list(
        set(
            [
                x
                for l in temp
                for x in l
                if x not in (ID2WORD[START_ID], ID2WORD[OOV_ID], "")
            ]
        )
    )
    elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=False)
    embeddings = elmo(temp, signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        embeddings = sess.run(embeddings)

    emb_matrix = np.zeros((vocabulary_size, 1024))
    for word, emb in zip(temp, embeddings.squeeze(axis=1)):
        i = WORD2ID[word]
        emb_matrix[i] = emb

    return emb_matrix


emb_matrix = _generate_elmo_embeddings()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


<IPython.core.display.Javascript object>

In [5]:
def build_model():
    inp = Input(shape=(max_words,))
    emb = Embedding(
        vocabulary_size, 1024, mask_zero=True, weights=[emb_matrix], trainable=True
    )(inp)
    x = LSTM(units=100)(emb)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    return model


model = build_model()
model.summary(line_length=200)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
input_1 (InputLayer)                                                                      [(None, 500)]                                                                   0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
embedding (Embedding)                                                                     (None, 500, 1024)                                                               5120000    

<IPython.core.display.Javascript object>

In [6]:
preprocessor = Preprocessor(maxlen=max_words, oov_token=ID2WORD[OOV_ID])
preprocessor.fit_on_texts(X_train + X_test)

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

<IPython.core.display.Javascript object>

In [7]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

model.fit(
    X_train2,
    y_train2,
    validation_data=(X_valid, y_valid),
    batch_size=batch_size,
    epochs=num_epochs,
)

Train on 24936 samples, validate on 64 samples
Epoch 1/3
24936/24936 [==============================] - 1103s 44ms/sample - loss: 0.4746 - acc: 0.7491 - val_loss: 0.2278 - val_acc: 0.9219
Epoch 2/3
24936/24936 [==============================] - 1361s 55ms/sample - loss: 0.2395 - acc: 0.9037 - val_loss: 0.1981 - val_acc: 0.9219
Epoch 3/3
24936/24936 [==============================] - 1345s 54ms/sample - loss: 0.1710 - acc: 0.9353 - val_loss: 0.1947 - val_acc: 0.9219


<IPython.core.display.Javascript object>

In [8]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test accuracy:", scores[1])

Test accuracy: 0.88844


<IPython.core.display.Javascript object>